Assignment 17

In [595]:
import numpy as np
import cv2
import math
import sys
import time
import enum
import matplotlib.pyplot as plt
import networkx as nx
from scipy.spatial.transform import Rotation as R



class Beacon:
    def __init__(self, location, id):
        self.location = location
        self.id = id
    def __str__(self):
        return "Beacon {} at {}".format(self.id, np.transpose(self.location))
    def __repr__(self):
        return self.__str__()

class Scanner:
    def __init__(self, location, id):
        self.id = id
        self.location = location
        self.beacons = []
    def AddBeacon(self, beacon):
        self.beacons.append(beacon)



def GetSensorData(dataset):
    data = []
    with open(dataset, 'r') as f:
        data = f.readlines()
    # strip the new line character
    
    data = [x.strip() for x in data]

    scanners = []
    sensor = {}
    beaconCount = 0
    scannerCount = 0
    for i in range(len(data)):
        #print(data[i])
        if (data[i].__contains__('scanner')):
            sensor = Scanner(np.array([[0], [0], [0]]), scannerCount)
            scannerCount += 1
            scanners.append(sensor)
        elif (data[i] == ''):
            continue
        else:
            location = data[i].split(',')
            beacon = Beacon(np.array([[int(location[0])], [int(location[1])], [int(location[2])]]), beaconCount)
            beaconCount += 1
            sensor.AddBeacon(beacon)

    return scanners

# create a distance map between each beacon in a scanner report
def CreateDistanceMaps(scanner):
    distanceMaps = []
    for i in range(len(scanner.beacons)):
        distanceMaps.append([scanner.beacons[i] , []])
        for j in range(len(scanner.beacons)):
            if (i == j):
                continue
            else:
                distanceMaps[i][1].append([scanner.beacons[j], 
                    math.sqrt(
                          math.pow(scanner.beacons[i].location[0] - scanner.beacons[j].location[0], 2) 
                        + math.pow(scanner.beacons[i].location[1] - scanner.beacons[j].location[1], 2) 
                        + math.pow(scanner.beacons[i].location[2] - scanner.beacons[j].location[2], 2))])
        # Sort the distance map by distance
        distanceMaps[i][1].sort(key=lambda x: x[1])
    
    return distanceMaps

# check for each distancemap if an x amount of beacons are in a similar distance from each other
def CheckForBeacons(distanceMap, compairmap, x):
    count = 0
    for i in range(len(distanceMap)):
        for j in range(len(compairmap)):
            if (distanceMap[i][1] == compairmap[j][1]):
                #print("Found a match")
                count += 1
                if (count == x-1):
                    return True
    #print("match count: {}".format(count))
    return False


In [596]:
def FindConnections(DistanceMap):
    scannerConnections = []
    for i in range(len(DistanceMap)):
        for j in range(i+1, len(DistanceMap)):
            if (i == j):
                continue
            else:
                breakbool = False
                for k in range(len(DistanceMap[i][1])):
                    for l in range(len(DistanceMap[j][1])):
                        if( CheckForBeacons(DistanceMap[i][1][k][1], DistanceMap[j][1][l][1], 12)):
                            print(f'scanner {i} and scanner {j} have similar beacons : {DistanceMap[i][1][k][0].id} and {DistanceMap[j][1][l][0].id}')
                            breakbool = True
                            scannerConnections.append([i, j, k, l])
                    if (breakbool):
                        break
    return scannerConnections

def GraphConnections(scannerConnections):
    G = nx.Graph()
    for i in range(len(DistanceMap)):
        G.add_node(i)
    for i in range(len(scannerConnections)):
        G.add_edge(scannerConnections[i][0], scannerConnections[i][1])

    nx.draw(G, with_labels=True)
    plt.show()


In [597]:
def FindConnectionTo(scannerConnections, scanner):
    for i in range(len(scannerConnections)):
        if (scannerConnections[i][0] == scanner.id):
            return scannerConnections[i][1]
        elif (scannerConnections[i][1] == scanner.id):
            return scannerConnections[i][0]

def CreateTransformationMatrix(translationX = 0, translationY = 0, translationZ = 0):
    return np.array([
        [1, 0, 0, translationX],
        [0, 1, 0, translationY],
        [0, 0, 1, translationZ],
        [0, 0, 0, 1]])
def CreateRotationMatrixX(angle):
    return np.array([
        [1, 0, 0, 0],
        [0, math.cos(angle), -math.sin(angle), 0],
        [0, math.sin(angle), math.cos(angle), 0],
        [0, 0, 0, 1]])
def CreateRotationMatrixY(angle):
    return np.array([
        [math.cos(angle), 0, math.sin(angle), 0],
        [0, 1, 0, 0],
        [-math.sin(angle), 0, math.cos(angle), 0],
        [0, 0, 0, 1]])

def CreateRotationMatrixZ(angle):
    return np.array([
        [math.cos(angle), -math.sin(angle), 0, 0],
        [math.sin(angle), math.cos(angle), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])

# augment the rotation matrix to a translation matrix
def AugmentRotationMatrix(rotationMatrix):
    return np.array([
        [rotationMatrix[0][0], rotationMatrix[0][1], rotationMatrix[0][2], 0],
        [rotationMatrix[1][0], rotationMatrix[1][1], rotationMatrix[1][2], 0],
        [rotationMatrix[2][0], rotationMatrix[2][1], rotationMatrix[2][2], 0],
        [0, 0, 0, 1]])

def AugmentVector(vector):
    return np.array([
        [vector[0][0]],[ vector[0][1]], [vector[0][2]], [1]])

def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

def rotation_matrix_from_vectors_simple(vec1, vec2):
    matrix = np.zeros((4,4))
    for i in range(len(vec1[0])):
        for j in range(len(vec2[0])):
            print(abs(vec1[0][i]) - abs(vec2[0][j]))
            value = abs(vec1[0][i]) - abs(vec2[0][j])
            if (value < 0.00001 and value > -0.00001):
                if (vec1[0][i] == vec2[0][j]):
                    matrix[i][j] = 1
                else:
                    matrix[i][j] = -1
    return matrix
                

def FindTransformation(DistanceMap1, DistanceMap2):
    # get 2 beacons that are in the same distance from each other
    beaconOrigin = DistanceMap1[0]
    beacon1 = beaconOrigin.location - DistanceMap1[1][0][0].location
    beacon2 = beaconOrigin.location - DistanceMap1[1][1][0].location
    planeNormal = np.cross(np.transpose(beacon1)/np.linalg.norm(beacon1), np.transpose(beacon2)/ np.linalg.norm(beacon2))


    beaconOrigin2 = DistanceMap2[0]
    beacon1_2 = beaconOrigin2.location - DistanceMap2[1][0][0].location
    beacon2_2 = beaconOrigin2.location - DistanceMap2[1][1][0].location
    planeNormal_2 = np.cross(np.transpose(beacon1_2)/np.linalg.norm(beacon1_2), np.transpose(beacon2_2)/ np.linalg.norm(beacon2_2))
    # get the angle of 2 vectors around z axis beacon1 and beacon1_2

    print(f'beacon1: {beacon1}')

    Rotation = rotation_matrix_from_vectors(planeNormal_2, planeNormal)
    beacon1_2 = np.dot(Rotation, beacon1_2)
    print(f'beacon1_2: {beacon1_2}')

    if (beacon1_2[0]!= beacon1[0]):
        Rotation1 = np.array([
            [-1, 0, 0],
            [0, -1, 0],
            [0, 0, -1]])
        beacon1_2 = np.dot(Rotation1, beacon1_2)
        Rotation = np.dot(Rotation, Rotation1)

    """
    angle1 = math.atan2(beacon1[1], beacon1[0]) - math.atan2(beacon1_2[1], beacon1_2[0])
    Rotation2 = CreateRotationMatrixZ(angle1)
    Rotation = np.matmul(Rotation2, AugmentRotationMatrix( Rotation))
    beacon1_2 = np.dot(Rotation2, AugmentVector(np.transpose(beacon1_2)))
    
    angle2 = math.atan2(beacon1[1], beacon1[2]) - math.atan2(beacon1_2[1], beacon1_2[2])
    Rotation2 = CreateRotationMatrixX(angle2)
    
    Rotation = np.matmul(Rotation2, AugmentRotationMatrix( Rotation))
    beacon1_2 = np.dot(Rotation2, AugmentVector(np.transpose(beacon1_2)))
    angle3 = math.atan2(beacon1[0], beacon1[2]) - math.atan2(beacon1_2[0], beacon1_2[2])
    Rotation2 = CreateRotationMatrixY(angle3)
    Rotation = np.matmul(Rotation2, AugmentRotationMatrix( Rotation))
    beacon1_2 = np.dot(Rotation2, AugmentVector(np.transpose(beacon1_2)))
"""
    print(f'orgin: {beaconOrigin.location}')
    print(f'orgin2: {beaconOrigin2.location}')

    transformVector = beaconOrigin2.location - beaconOrigin.location
    transposeMatrix = CreateTransformationMatrix(transformVector[0], transformVector[1], transformVector[2])
    transformMatrix = AugmentRotationMatrix( Rotation)

    print(f'Rotation: {transformMatrix}')
    print(f'beacon1_2: {beacon1_2}')


    beacon1_2 =transformVector + beacon1_2
    print(f'beacon1_2: {beacon1_2}')
    # apply the rotation to all beacons in the distance map 2
    # and check if the location of the beacon is the same as in map1
    print(DistanceMap2[1])
    print(DistanceMap1[1])
    for i in range(len(DistanceMap2[1])):
            DistanceMap2[1][i][0].location = transformMatrix.dot(np.dot(transposeMatrix, AugmentVector(np.transpose(DistanceMap2[1][i][0].location))))
            #print(f'DistanceMap2[1][{i}][0].location: {DistanceMap2[1][i][0].location}')
            #print(f'DistanceMap1[1][{i}][0].location: {DistanceMap1[1][i][0].location}')
            DistanceMap1[1][i][0].location = AugmentVector (np.transpose(DistanceMap1[1][i][0].location))

            if (DistanceMap1[1][i][0].location[0] != DistanceMap2[1][i][0].location[0] or DistanceMap1[1][i][0].location[1] != DistanceMap2[1][i][0].location[1] or DistanceMap1[1][i][0].location[2] != DistanceMap2[1][i][0].location[2]):
                print(f'beacon: {DistanceMap1[1][i][0].id}')
                print(f'DistanceMap2[1][{i}][0].location: {DistanceMap2[1][i][0].location}')
                print(f'DistanceMap1[1][{i}][0].location: {DistanceMap1[1][i][0].location}')
                print('not equal')

                print(f'Error: beacon {i} location is not the same')
    
    return transformMatrix

    
    
    

In [598]:
scanners = GetSensorData('./data/aoc19_test2.txt')

DistanceMaps = []
for i in range(len(scanners)):
    DistanceMaps.append([scanners[i] ,CreateDistanceMaps(scanners[i])])

scannerConnections = FindConnections(DistanceMaps)
#GraphConnections(scannerConnections)
#print(DistanceMaps[0])
#print(scannerConnections)
i , j , k , l = scannerConnections[3]
#print(DistanceMaps[i][1][k])
FindTransformation(DistanceMaps[i][1][k], DistanceMaps[j][1][l])

scanner 0 and scanner 1 have similar beacons : 0 and 28
scanner 1 and scanner 3 have similar beacons : 31 and 78
scanner 1 and scanner 4 have similar beacons : 27 and 105
scanner 2 and scanner 4 have similar beacons : 50 and 115
beacon1: [[ 71]
 [-64]
 [-16]]
beacon1_2: [[-71.]
 [ 64.]
 [ 16.]]
orgin: [[649]
 [640]
 [665]]
orgin2: [[ 808]
 [-476]
 [-593]]
Rotation: [[-0.80852064  0.14252865  0.57094654  0.        ]
 [ 0.2376356  -0.80852064  0.53835276  0.        ]
 [-0.53835276 -0.57094654 -0.61983575  0.        ]
 [ 0.          0.          0.          1.        ]]
beacon1_2: [[ 71.]
 [-64.]
 [-16.]]
beacon1_2: [[  230.]
 [-1180.]
 [-1274.]]
[[Beacon 120 at [[ 872 -547 -609]], 96.91749068150702], [Beacon 108 at [[ 927 -485 -438]], 195.61952867748147], [Beacon 109 at [[ 408  393 -506]], 960.5883613702593], [Beacon 110 at [[ 466  436 -512]], 977.3786369672707], [Beacon 123 at [[ 839 -516  451]], 1045.2258129227387], [Beacon 126 at [[ 30 -46 -14]], 1060.860499783077], [Beacon 107 at [[ 8

C:\Users\IANARB~1\AppData\Local\Temp/ipykernel_19196/3957883865.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array([


array([[-0.80852064,  0.14252865,  0.57094654,  0.        ],
       [ 0.2376356 , -0.80852064,  0.53835276,  0.        ],
       [-0.53835276, -0.57094654, -0.61983575,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [599]:
if (False):
    scanners = GetSensorData('./data/aoc19.txt')

    DistanceMap = []
    for i in range(len(scanners)):
        DistanceMap.append([scanners[i] ,CreateDistanceMaps(scanners[i])])

    scannerConnections = FindConnections(DistanceMap)
    GraphConnections(scannerConnections)